In [190]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import random


In [1]:

CONFIG = {
    'intents': {
        'hello': {
            'requests': ['привет', 'добрый день', 'здравствуй', 'как дела'],
            'responses': ['Привет, босс', 'давай говори чего хочешь'],
        },
        'name': {
            'requests': ['как твоё имя', 'как тебя зовут'],
            'responses': ['меня зовут Саламандра'],
        },
        'uwhere': {
            'requests': ['ты где', 'куда подевалась'],
            'responses': ['я тут, ты меня спрашиваешь, что-ли?', 'тута я'],
        },
        'abuse': {
            'requests': ['плохо', 'нехорошо', 'нехорошая', 'дура', 'коза', 'бестолковая', 'заткнись', 'задолбала'],
            'responses': ['на себя посмотри', 'а чё сразу ругаться та', 'ну обидно же', 'я больше так не буду'],
        },
        'praise': {
            'requests': ['красава', 'молодец', 'хорошо', 'отлично', 'хорошая', 'приятно поговорить', 'спасибо', 'благодарю'],
            'responses': ['спасибо', 'мне очень приятно', 'стараюсь'],
        },
        'ctime': {
            'requests': ['текущее время', 'сколько время', 'сколько времени', 'который час'],
            'responses': [''],
            'actions': {'время': 'ctime'}
        },
        'age': {
            'requests': ['сколько тебе лет', 'твой возраст'],
            'responses': ['я еще молода'],
            'actions': {'возраст лет': 'my_age'}
        },
        'whois': {
            'requests': ['что такое', 'кто такой'],
            'responses': [''],
            'actions': {'такое': 'whois'}
        },
        'translate': {
            'requests': ['переведи', 'как будет по-английски'],
            'responses': [''],
            'actions': {'': 'translate'}
        },
        'music': {
            'requests': ['радио', 'включи', 'послушать', 'музыку'],
            'responses': ['включаю', 'как скажешь', 'сама с удовольствием послушаю', 'хорошо', 'а га', ''],
            'actions': {
                'like fm': 'radio_like_fm',
                'лайк фм': 'radio_like_fm',
                'офис lounge': 'radio_office_lounge',
                'office lounge': 'radio_office_lounge',
                'чилстеп': 'radio_chillstep',
                'chillstep': 'radio_chillstep',
                'чипльдук': 'radio_chip',
                'мою музыку': 'music_my',
            },
            'reask': ['не поняла что именно', 'уточни', 'что ты хочешь послушать']
        },
        'browse': {
            'requests': ['открой'],
            'responses': ['открываю', 'как скажешь', 'интересно что же там', ],
            'actions': {
                'яндекс музыку': 'music_yandex',
            }
        },
        'start': {
            'requests': ['открой программу'],
            'responses': ['открываю', 'есть', ],
            'actions': {
                'ноутпад': 'notepad',
            }
        },
        'find': {
            'requests': ['найди', 'спроси у', 'загугли', 'поищи'],
            'responses': ['пошла искать', 'уже ищу', 'секундочку', 'это где-то здесь', 'что-то нашла'],
            'actions': {
                'яндексе': 'browse_yandex',
                'википедии': 'wikipedia',
                'гугла': 'browse_google',
                'загугли': 'browse_google',
                'youtube': 'youtube',
                'ютюбе': 'youtube',
            }
        },
        'bye': {
            'requests': ['пока', 'до свидания', 'прощай', 'спокойной ночи'],
            'responses': ['Счаст ливо', 'Еще увидимся', 'Если что, я тут. Возвращайтесь'],
            'actions': []
        },
    },
    'failure_phrases': [
        'Вот это сейчас что было?',
        'Что-то не понятно',
        'А можно как-то попроще выразиться?',
        'К сожалению, я не смогу помочь вам с этим вопросом',
        'Слишком сложно для меня',
        'Здесь наши полномочия всё',
        'Моя твоя не понимай',
        'Уточните вопрос, пожалуйста',
        'Ничего не поняла, но оочень интересно',
        'Вас людей не всегда поймешь',
        'Я не совсем тебя поняла',
        'Как это понимать?',
        'Но-но, Полегче!',
        'Запуталась я с вами совсем.',
        'Меня такому не учили',
        'Может мы забудем, что ты сказал?',
        'Ещё ра зочек можно?',
        'Мне не понятно.',
        'Я правильно интерпретирую семантику вопроса, но полностью игнорирую его суть',
        'Прости, чёт не вкурила',
        'ты пытаешься меня запутать?!',
        'Туплю... повто ри',
        'Это точно правильные слова?',
        'Я не поняла твоих намерений',
        'Я же всего лишь бот. Скажи попроще',
        'Я поняла, что я не поняла',
        'Со рян не поняла',
        'Сначала я ничего не поняла, а потом я тоже ничего не поняла',
    ],
    "tbr": (
        'хочу',
        'хочется',
        'что-то',
        'скажи',
        'расскажи',
        'покажи',
        'сколько',
        'произнеси',
        'слушай',
        'пожалуйста',
        'если можно',
        'если можешь',
        'ка мне',
        'потому что',
        "ну",
        'ты',
        'сейчас',
        'так',
    ),
    'alias': ('Саламандра', 'l\'amant', 'помандер', 'salamander', 'саламандр', 'мантра'),
    'whazzup': {
        'offline': ' работает в оф лайн режиме. Поэтому говори очень разборчиво',
        'online': ' слушает',
    }
}


In [236]:
import pymorphy2  # $ pip install pymorphy2
  
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS
  
words = "Однако я так и не смог закончить".split()
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
print(*[word for word in words if pos(word) not in functors_pos])


я смог закончить
